In [ ]:
### 載入套件
import requests
from bs4 import BeautifulSoup as bs 

In [ ]:
### 取得 首頁 source code
url = "https://www.nownews.com/cat/column/"

headers= {
    "user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36"
}

res = requests.get(url,headers=headers)
soup = bs(res.text,"lxml")

In [ ]:
### 抓取 1 ~ 10 篇文章連結
links = []

# 抓取 1~5 篇
for ele in soup.select("div.sliderBlk a"):
    url = ele["href"]
    links.append(url)
    
# 抓取 6~10 篇
for ele in soup.select("div.leftCol div.listBlk a"):
    links.append(ele["href"])
    
for url in links:
    print(url)

In [ ]:
### 抓取本文資料 -> 收集成一個 function 
def parsing_content_data(content_soup):
    
    # 標題
    title = content_soup.select("div.titleBlk h1")[0].text

    # 分類
    # \n -> 換行符號
    category = content_soup.select("div.breadCrumbBlk")[0].text
    category = category.strip().replace("\n\n"," > ")

    # 時間
    # time = content_soup.select("div.titleBlk p.time")[0].text.strip()
    # 時間格式更新
    raw_time = content_soup.select("div.titleBlk p.time")[0].text.strip()
    time = raw_time.split("｜")[0]

    # 內容
    article = content_soup.select("div.leftCol article")[0]

    # 拔除 div.ad-blk1 , ul.related 標籤
    if len(article.select("div.ad-blk1"))>0:
        article.select("div.ad-blk1")[0].extract()
    if len(article.select("ul.related")) >0:
        article.select("ul.related")[0].extract()

    content = article.text.strip()
    
    return {
        "title" : title,
        "category" : category,
        "time"  : time,
        "content" : content
    }


def get_source_code(url):
    headers= {
        "user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36"
    }
    res = requests.get(url,headers=headers)
    soup = bs(res.text,"lxml")
    
    return soup



url3 = links[8]
print(url3)
soup3 = get_source_code(url3)
parsing_content_data(soup3)

In [ ]:
### 抓取 1 ~ 10 篇文章本文
dataList = []
for url in links:
    content_soup = get_source_code(url)
    data = parsing_content_data(content_soup)
    dataList.append(data)
    
    print("{} is ok.".format(url))
    
print("Done.")

In [ ]:
### 資料輸出成檔案 (資料落地)
for data in dataList:

    file_name = "sample/{}.txt".format(data["title"])
    with open(file_name,"w",encoding="utf8") as out_file:
        
        record = ""
        
        # 資料全部讀取完 , 再一次寫出檔案
        for key in data:
            record += "{}:{}\n".format(key , data[key])
            
        out_file.write(record)

        # 每讀一段資料 , 就寫出檔案一次 (不佳)
        # for key in data:
        #     msg = "{}:{}\n".format(key , data[key])
        #     out_file.write(msg)

In [ ]:
### 透過 API 抓 JSON 資料

# 1. 抓取 txtPageNo , 取得 1st 參數
pid = soup.select("input#txtPageNo")[0]["value"]
print(pid)

# 2. 向 API 發 requests , 取得文章列表資料
api = "https://www.nownews.com/nn-client/api/v1/cat/column/?pid={}".format(pid)
headers = {
    "user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36"
}

res = requests.get(api,headers=headers)

# response 用文字格式輸出
# print(res.text)

# response 用 json 格式輸出
data = res.json()

links2 = []

for ele in data["data"]["newsList"]:
    print(ele["postTitle"])
    # print(ele["postUrl"])
    
    url = "https://www.nownews.com" + ele["postUrl"]
#     print(url)
    links2.append(url)

    print("-"*80)
    
# ele 停留在最後一筆資料
pid = ele["id"]


# 3. 再透過 API 的資料 , 取得下一次的 requests 參數
for i in range(1,4):
    api = "https://www.nownews.com/nn-client/api/v1/cat/column/?pid={}".format(pid)
    headers = {
        "user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36"
    }

    res = requests.get(api,headers=headers)
    
    data = res.json()
    
    for ele in data["data"]["newsList"]:
        url = "https://www.nownews.com" + ele["postUrl"]
        links2.append(url)
        
    pid = ele["id"]
    
print("Done.")